In [11]:
import pandas as pd
from pandasql import sqldf

In [14]:
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
!pip install openpyxl

In [ ]:
!pip install pandasql

In [20]:
bank_data = pd.read_excel("C:\\Users\\Марина\\1. МФТИ\\Аналитика\\bank_account_data.xlsx")

In [37]:
#Опишите имеющиеся колонки в данных - сделайте аналог describe в pandas для колонок.

ID = pysqldf("""SELECT    
        COUNT(Cust_Id),
        AVG(Cust_Id),
        MIN(Cust_Id),
        MAX(Cust_Id)     
        FROM bank_data""")

Purchase = pysqldf("""SELECT    
        COUNT(Purchase),
        AVG(Purchase),
        MIN(Purchase),
        MAX(Purchase)     
        FROM bank_data""")

Cheq_Bal = pysqldf("""SELECT    
        COUNT(Cheq_Bal),
        AVG(Cheq_Bal),
        MIN(Cheq_Bal),
        MAX(Cheq_Bal)     
        FROM bank_data""")

Sav_Bal = pysqldf("""SELECT    
        COUNT(Sav_Bal),
        AVG(Sav_Bal),
        MIN(Sav_Bal),
        MAX(Sav_Bal)     
        FROM bank_data""")

Debt_Bal = pysqldf("""SELECT    
        COUNT(Debt_Bal),
        AVG(Debt_Bal),
        MIN(Debt_Bal),
        MAX(Debt_Bal)     
        FROM bank_data""")

Joint_Acct = pysqldf("""SELECT    
        COUNT(Joint_Acct),
        AVG(Joint_Acct),
        MIN(Joint_Acct),
        MAX(Joint_Acct)     
        FROM bank_data""")

Age = pysqldf("""SELECT    
        COUNT(Age),
        AVG(Age),
        MIN(Age),
        MAX(Age)     
        FROM bank_data""")

print(ID, Purchase, Cheq_Bal, Sav_Bal, Debt_Bal, Joint_Acct, Age, sep="\n")

#describe = pd.concat([Cust_ID, Purchase, Cheq_Bal, Sav_Bal, Debt_Bal, Joint_Acct, Age])
#describe['index'] = ['Cust_ID', 'Purchase', 'Cheq_Bal', 'Sav_Bal', 'Debt_Bal', 'Joint_Acct', 'Age']
#describe.set_index('index')

   COUNT(Cust_Id)  AVG(Cust_Id)  MIN(Cust_Id)  MAX(Cust_Id)
0             200         100.5             1           200
   COUNT(Purchase)  AVG(Purchase)  MIN(Purchase)  MAX(Purchase)
0              200          0.195              0              1
   COUNT(Cheq_Bal)  AVG(Cheq_Bal)  MIN(Cheq_Bal)  MAX(Cheq_Bal)
0              200      2145.0935         644.31        4437.44
   COUNT(Sav_Bal)  AVG(Sav_Bal)  MIN(Sav_Bal)  MAX(Sav_Bal)
0             200   22084.42895       6558.91      39132.63
   COUNT(Debt_Bal)  AVG(Debt_Bal)  MIN(Debt_Bal)  MAX(Debt_Bal)
0              200      8638.3696            0.0        20559.7
   COUNT(Joint_Acct)  AVG(Joint_Acct)  MIN(Joint_Acct)  MAX(Joint_Acct)
0                200            0.295                0                1
   COUNT(Age)  AVG(Age)  MIN(Age)  MAX(Age)
0         200    38.535        18        60


In [ ]:
#Посчитайте совокупный объем средств у клиентов в банке - сумма по всем балансам.
pysqldf("""SELECT 
        Cust_ID,
        Cheq_Bal+Sav_Bal+Debt_Bal AS TOTAL_bal
        FROM bank_data""")


In [ ]:
#Выведите только тех клиентов, которые приобрели ипотеку (колонка mortgage).
pysqldf("""SELECT * 
        FROM bank_data
        WHERE Purchase = 1        
        """)

In [ ]:
#Какой совокупный объем средств в среднем у клиентов, которые приобрели ипотеку и
# у клиентов, которые её не приобретали?

query = """SELECT 
            SUM(CASE WHEN Purchase=1 THEN Cheq_Bal+Sav_Bal+Debt_Bal END) / COUNT(CASE WHEN Purchase=1 THEN 1 END) as Purchase,
            SUM(CASE WHEN Purchase=0 THEN Cheq_Bal+Sav_Bal+Debt_Bal END) / COUNT(CASE WHEN Purchase=0 THEN 1 END) as Non_Purchase
            FROM bank_data
        """

pysqldf(query)


In [47]:
#Есть ли разница в среднем по каждому из типов баланса у клиентов, которые приобрели ипотеку 
# и у клиентов, которые её не приобретали?

query = """SELECT 
            SUM(CASE WHEN Purchase=1 THEN Cheq_Bal END) / COUNT(CASE WHEN Purchase=1 THEN 1 END) -
            SUM(CASE WHEN Purchase=0 THEN Cheq_Bal END) / COUNT(CASE WHEN Purchase=0 THEN 1 END) as Cheq_Bal_diff,
            
            SUM(CASE WHEN Purchase=1 THEN Sav_Bal END) / COUNT(CASE WHEN Purchase=1 THEN 1 END) -
            SUM(CASE WHEN Purchase=0 THEN Sav_Bal END) / COUNT(CASE WHEN Purchase=0 THEN 1 END) as Sav_Bal_diff,
            
            SUM(CASE WHEN Purchase=1 THEN Debt_Bal END) / COUNT(CASE WHEN Purchase=1 THEN 1 END) -
            SUM(CASE WHEN Purchase=0 THEN Debt_Bal END) / COUNT(CASE WHEN Purchase=0 THEN 1 END) as Debt_Bal_diff
            
            FROM bank_data
        """

pysqldf(query)

,Cheq_Bal_diff,Sav_Bal_diff,Debt_Bal_diff
0,761.344275,9370.354545,-6172.688466


In [ ]:
# Выберите самого возрастного получателя ипотеки.
pysqldf("""SELECT Cust_Id, 
        MAX(Age)
        FROM bank_data    
        """)


In [ ]:
#Выберите клиента, который раньше всех получил ипотеку.
pysqldf("""SELECT Cust_Id, 
        MIN(Date)
        FROM bank_data    
        """)


In [71]:
# Создайте любую базу данных в sqlite3, состоящую из 5 колонок, 
# одна из колонок обязательно должна быть типа REAL. (Названия колонок любые, какие захотите)
# Добавьте 3 полностью заполненные строки в вашу БД.

import sqlite3
con = sqlite3.connect("tutorial1_DB.db")
cur = con.cursor()
cur.execute("CREATE TABLE movie(ID, title, year, score real, director)")

In [73]:
cur.execute("""
    INSERT INTO movie VALUES
        (1, 'Monty Python and the Holy Grail', 1975, 8.2, 'Terry Gilliam'),
        (2, 'And Now for Something Completely Different', 1971, 7.5, 'Terry Gilliam'),
        (3, '12 Monkeys', 1995, 8.0, 'Terry Gilliam')
""")

movie_data = cur.execute("""SELECT * 
        FROM movie
        """)
movie_data.fetchall()

[(1, 'Monty Python and the Holy Grail', 1975, 8.2, 'Terry Gilliam'),
 (2, 'And Now for Something Completely Different', 1971, 7.5, 'Terry Gilliam'),
 (3, '12 Monkeys', 1995, 8.0, 'Terry Gilliam')]

In [ ]:
con.execute("""
            DELETE FROM movie
            WHERE 
            score <> (select MAX(score) from movie)
            AND 
            score <> (select MIN(score) from movie)
            """)

In [70]:
cur.execute("""DROP TABLE movie;""")
con.close()